# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv


# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nuuk,64.1835,-51.7216,-8.66,67,100,7.20,GL,1706670098
1,1,rio grande,-32.0350,-52.0986,20.81,88,15,2.79,BR,1706670101
2,2,palapag,12.5458,125.1147,26.68,77,97,6.71,PH,1706670104
3,3,ambon,-3.6954,128.1814,32.08,62,40,3.09,ID,1706670108
4,4,bethel,41.3712,-73.4140,1.31,77,100,2.50,US,1706670111


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map = city_data_df.hvplot.points(x='Lng', y='Lat', geo=True, c='City',tiles='OSM', size='Humidity', hover_cols=['City'], alpha =0.5)
map = map.opts(title='City Humidity Map', xlabel='Longitude', ylabel='Latitude')
# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)].copy()

# Drop any rows with null values
ideal_conditions_df.dropna(inplace=True)

# Display sample data
ideal_conditions_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,port elizabeth,-33.9180,25.5701,21.29,88,0,1.54,ZA,1706670242
347,347,santarem,-2.4431,-54.7083,25.07,100,0,2.06,BR,1706671024
380,380,kyaikkami,16.0800,97.5675,25.35,72,0,2.47,MM,1706671258
426,426,piacabucu,-10.4056,-36.4344,25.06,79,0,3.16,BR,1706671393
446,446,timaru,-44.4000,171.2500,21.15,77,0,2.68,NZ,1706671450


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
51,port elizabeth,ZA,-33.9180,25.5701,88,
347,santarem,BR,-2.4431,-54.7083,100,
380,kyaikkami,MM,16.0800,97.5675,72,
426,piacabucu,BR,-10.4056,-36.4344,79,
446,timaru,NZ,-44.4000,171.2500,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{lon},{lat},{radius}"
    bias = f"proximity:{lon},{lat}"
    params = {
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key 
    }
    

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    params["categories"] = f"accommodation.hotel"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']}  nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth  nearest hotel: Waterford Hotel
santarem  nearest hotel: Prático Hotel
kyaikkami  nearest hotel: ရှုခင်းသာ
piacabucu  nearest hotel: O Leão
timaru  nearest hotel: The Grosvenor Hotel
mapastepec  nearest hotel: Hotel San Diego
arauca  nearest hotel: Hotel Nova Park
zayed city  nearest hotel: L'Arabia hotel
vila velha  nearest hotel: Hotel Prainha


,City,Country,Lat,Lng,Humidity,Hotel Name
51,port elizabeth,ZA,-33.9180,25.5701,88,Waterford Hotel
347,santarem,BR,-2.4431,-54.7083,100,Prático Hotel
380,kyaikkami,MM,16.0800,97.5675,72,ရှုခင်းသာ
426,piacabucu,BR,-10.4056,-36.4344,79,O Leão
446,timaru,NZ,-44.4000,171.2500,77,The Grosvenor Hotel
449,mapastepec,MX,15.4333,-92.9000,63,Hotel San Diego
483,arauca,CO,7.0847,-70.7591,78,Hotel Nova Park
492,zayed city,AE,24.3768,54.5890,56,L'Arabia hotel
505,vila velha,BR,-20.3297,-40.2925,94,Hotel Prainha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
#%%capture --no-display

# Configure the map plot
# Configure the map plot
map = hotel_df.hvplot.points(x='Lng', y='Lat', geo=True, c='City', tiles='OSM', size='Humidity', alpha=0.5, hover_cols=['City', 'Hotel Name'])

map = map.opts(title='City Humidity Map', xlabel='Longitude', ylabel='Latitude')


# Display the map
map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)